In [ ]:
!pip install langdetect
!pip install googletrans==4.0.0-rc1
!pip install num2words
!pip install ftfy
!pip install nltk
!pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=727ff52f409247302327bfa5894d95a6c9c476b13a49dec9202f02efa39173df
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.3 MB/s eta 

In [ ]:
!pip install keras
!pip install tensorflow

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import nltk
from langdetect import detect
from googletrans import Translator
from num2words import num2words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import ftfy
import joblib
from sklearn.base import BaseEstimator, TransformerMixin

# Descargar recursos de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Paso 1: Función para detectar idioma y traducir si es necesario
def translate_text(text):
    translator = Translator()
    detected_lang = detect(text)
    if detected_lang != 'es':
        try:
            translation = translator.translate(text, src=detected_lang, dest='es')
            return translation.text
        except Exception:
            return text
    return text

# Paso 2: Función para limpieza de texto
def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres no ASCII
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    # Eliminar signos de puntuación
    text = re.sub(r'[^\w\s]', '', text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Paso 3: Corrección de errores de codificación
def fix_encoding(text):
    return ftfy.fix_text(text)

# Paso 4: Conversión de números a palabras
def convert_numbers(text):
    return ' '.join([num2words(word, lang='es') if word.isdigit() else word for word in text.split()])

# Paso 5: Tokenización de palabras
def tokenize_text(text):
    return word_tokenize(text)

# Paso 6: Aplicar stemming y lematización
def stem_and_lemmatize(tokens):
    stemmer = SnowballStemmer('spanish')
    lemmatizer = WordNetLemmatizer()
    stems = [stemmer.stem(token) for token in tokens]
    lemmas = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    # Concatenar stems y lemmas para mayor cobertura
    return stems + lemmas

# Paso 7: Preprocesamiento completo del texto
def preprocess_text(text):
    # Traducción de texto si es necesario
    text = translate_text(text)
    # Limpieza básica
    text = clean_text(text)
    # Corrección de codificación
    text = fix_encoding(text)
    # Conversión de números a palabras
    text = convert_numbers(text)
    # Tokenización
    tokens = tokenize_text(text)
    # Stemming y lematización
    processed_tokens = stem_and_lemmatize(tokens)
    # Unir tokens procesados en una cadena
    return ' '.join(processed_tokens)

# Clase personalizada para el preprocesamiento de texto
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Aplicar el preprocesamiento personalizado a cada texto
        return X.apply(preprocess_text)

# Paso 8: Cargar los datos
data_path = '/content/ODScat_345.xlsx'
ods_data = pd.read_excel(data_path)

# Paso 9: Verificar la estructura de los datos
print(f"El archivo tiene {ods_data.shape[0]} filas y {ods_data.shape[1]} columnas.")
print("\nPrimeros registros:")
print(ods_data.head())

# Paso 10: Verificar si existen duplicados y eliminarlos
ods_data = ods_data.drop_duplicates()

# Paso 11: Separar características (X) y etiquetas (y)
X_data = ods_data['Textos_espanol']
y_data = ods_data['sdg']

# Paso 12: División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

# Paso 13: Definir el pipeline con el preprocesador personalizado, TF-IDF y Naive Bayes
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()), # Transformador personalizado para limpieza
    ('vectorizer', TfidfVectorizer()),  # Utiliza TfidfVectorizer para transformar el texto en características numéricas
    ('classifier', MultinomialNB())     # Utiliza el clasificador Naive Bayes
])

# Paso 14: Entrenamiento del modelo
pipeline.fit(X_train, y_train)

# Paso 15: Predicción y evaluación
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nExactitud del modelo: {accuracy:.4f}")
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))

# Paso 16: Guardar el modelo entrenado para uso en la API
joblib.dump(pipeline, 'modelo_entrenado_completo.joblib')


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


# Paso 21: Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Paso 22: Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.classes_)
disp.plot(cmap='Blues', values_format='d')
plt.title('Matriz de Confusión del Modelo')
plt.xlabel('Etiqueta Predicha')
plt.ylabel('Etiqueta Verdadera')
plt.show()


Prediccion

In [ ]:
import pandas as pd
from joblib import load

# Cargar el modelo entrenado
pipeline = load('modelo_entrenado_completo.joblib')

# Cargar los datos de prueba
test_data_path = '/content/TestODScat_345.xlsx'
test_data = pd.read_excel(test_data_path)

# Asegurarse de que la columna con los textos se llame 'Textos_espanol'
# Si no es así, ajusta el nombre de la columna aquí.
if 'Textos_espanol' not in test_data.columns:
    print("Error: La columna 'Textos_espanol' no se encontró en el archivo de prueba.")
else:
    # Realizar la predicción con el modelo cargado
    predictions = pipeline.predict(test_data['Textos_espanol'])

    # Agregar las predicciones al DataFrame
    test_data['Predicciones'] = predictions

    # Mostrar los resultados
    print(test_data[['Textos_espanol', 'Predicciones']])

    # Guardar el DataFrame con las predicciones en un archivo (opcional)
    test_data.to_excel('resultados_predicciones.xlsx', index=False)
